In [1]:
import habitat
from habitat.config.default import get_config
%matplotlib inline
import matplotlib.pyplot as plt
from habitat.tasks.nav.shortest_path_follower import ShortestPathFollower
from typing import Any
import numpy as np

In [2]:
config = habitat.get_config(config_paths="/data/data/challenge_pointnav2020.local.rgbd.yaml")
# config = get_config()

In [3]:
config.defrost()

config.DATASET.DATA_PATH = '/data/data/datasets/pointnav/habitat-test-scenes/v1/test/test.json.gz'
# config.SIMULATOR.TURN_ANGLE = 0.5
config.ENVIRONMENT.MAX_EPISODE_STEPS = 100000
config.TASK.TOP_DOWN_MAP.MAX_EPISODE_STEPS = 100000
config.DATASET.SCENES_DIR = '/data/data/scene_datasets'
config.TASK.MEASUREMENTS.append("DISTANCE_TO_GOAL")

config.TASK.SENSORS = ["POINTGOAL_WITH_GPS_COMPASS_SENSOR", "COMPASS_SENSOR", "GPS_SENSOR", "POINTGOAL_SENSOR",]
config.TASK.POINTGOAL_WITH_GPS_COMPASS_SENSOR.DIMENSIONALITY = 3
config.TASK.POINTGOAL_WITH_GPS_COMPASS_SENSOR.GOAL_FORMAT = "CARTESIAN"

config.TASK.POINTGOAL_SENSOR.DIMENSIONALITY = 3
config.TASK.POINTGOAL_SENSOR.GOAL_FORMAT = "CARTESIAN"

config.TASK.GPS_SENSOR.DIMENSIONALITY = 3

config.freeze()

In [4]:
class SimpleRLEnv(habitat.RLEnv):
    def get_reward_range(self):
        return [-1, 1]

    def get_reward(self, observations):
        return 0

    def get_done(self, observations):
        return self.habitat_env.episode_over

    def get_info(self, observations):
        return self.habitat_env.get_metrics()

In [5]:
env = SimpleRLEnv(config)

2020-03-20 08:23:24,352 Initializing dataset PointNav-v1
2020-03-20 08:23:24,365 initializing sim Sim-v0
2020-03-20 08:23:27,928 Initializing task Nav-v0


In [6]:
observations = env.reset()

In [7]:
print(observations.keys())

dict_keys(['rgb', 'depth', 'pointgoal_with_gps_compass', 'compass', 'gps', 'pointgoal'])


In [9]:
episode = env.current_episode

goals = episode.goals
cur_goal = goals[0]
position = cur_goal.position

In [10]:
state = env.habitat_env._sim.get_agent_state()

state.position = position

env.habitat_env._sim.set_agent_state(state.position, state.rotation, 0, False)

True

In [11]:
episode = env.current_episode

goals = episode.goals
cur_goal = goals[0]
position = cur_goal.position

In [12]:
observations = env.step('TURN_LEFT')[0]

env.habitat_env._sim.get_agent_state().position

array([-4.1612463,  0.0320512,  6.3028803], dtype=float32)

In [18]:
start_position = env.current_episode.start_position
cur_position = env.habitat_env._sim.get_agent_state().position

In [20]:
print('finish position: ', position)
print('start position: ', start_position)
print('gps: ', observations['gps'])
print('pointgoal: ', observations['pointgoal'])
print('data: ', observations['pointgoal_with_gps_compass'])
print('cur position: ', cur_position)

finish position:  [-4.124889850616455, 0.00991860032081604, 6.324680328369141]
start position:  [0.11632350087165833, 0.20138363540172577, 16.983539581298828]
gps:  [10.107671   -0.16933243  5.4962792 ]
pointgoal:  [10.06576574 -0.19146504  5.502684  ]
data:  [-0.04235396 -0.0221326  -0.00178203]
cur position:  [-4.1612463  0.0320512  6.3028803]
